# Temat 1. Goofspiel (Game Of Pure Strategy)

## Na czym polega Goofspiel?

Goofspiel, czyli Gra Czystej Strategii (GOPS), to gra dla dwóch osób (przy naszej wersji). Wykorzystuje standardową talię 52 kart, z której odrzucane są wszystkie karty jednego koloru. Karty jednego koloru otrzymuje jeden gracz, karty drugiego koloru otrzymuje drugi gracz, a karty pozostałego koloru są tasowane i umieszczane zakryte na środku. Karty mają wartości od niskiej do wysokiej: as = 1, 2, 3, ..., 10, walet = 11, dama = 12, król = 13.

Runda polega na odkryciu następnej karty z środkowej sterty i "postawieniu" na nią przez obu graczy, którzy wybierają jedną kartę, a następnie jednocześnie ją pokazują. Gracz, który pokaże kartę o wyższej wartości, wygrywa wartość odkrytej karty. Jeśli obaj gracze pokażą tę samą kartę, punkty są dzielone między nich. Te trzy karty są następnie odrzucane. Gra kończy się po 13 rundach, a zwycięzcą jest osoba, która zdobyła najwięcej punktów (do wygranej potrzeba 46 punktów lub więcej).

Mechanika gry jest prosta, ale strategia już nie. Przykładowo, król jest odkrytą kartą w pierwszej rundzie. Załóżmy, że postawisz jeden (tzn. as). Kiedy odkrywasz swoją kartę, okazuje się, że twój przeciwnik postawił swojego króla, wygrywając 13 punktów. Cieszysz się z tego wyniku, bo teraz masz 12 dodatkowych punktów do "stawki", co powinno z nawiązką zrekompensować stracone 13 punktów. Jednak ryzykujesz stawiając tylko jeden: jeśli twój przeciwnik postawiłby dwa lub trzy, to wygrałby 13 punktów za niemal żadną cenę. Grając w grę, próbujesz przewidzieć ruchy przeciwnika, podczas gdy on próbuje przewidzieć twoje: zastanawiasz się nad takimi kwestiami jak "mój przeciwnik prawdopodobnie zagra X, więc powinienem zagrać Y, ale on może to przewidzieć i zagrać Z, aby pokonać Y, więc może powinienem zagrać W. Chcemy spojrzeć na ten problem z wyższej perspektywy i obliczyć optymalne zagrania w rozumieniu teorii gier. 

## GOPS jako problem liniowy

Moim zadaniem było przeformułowanie problemu z teorii gier na zagadnienie programowania liniowego i odpowiednie zaprogramowanie, by uzyskać rezultaty. Wykorzystałem metodę opisaną w *Rhoads, G. C.; Bartholdi, L. (2012). "Computer Solution to the Game of Pure Strategy". Games. 3 (4): 150–156.*

Napisany program pozwala wygenerować strategię dla pierwszego ruchu, co było głównym celem zadania, jak również dla wszystkich możliwych podgier, co byłoby niezbędne, gdybyśmy chcieli przeprowadzić całą rozgrywkę przy użyciu optymalych strategii. 

Skupiłem się na przykładzie dla **N = 6**. Program działa dla dowolnego N, jednak dalsze obliczenia przekraczają możliwości platformy. 

## Implementacja 

Kluczowe zadanie wykonuje funkcja *game_size_k()*, która jest odpowiedzialna za generowanie ostatecznych wyników. Do obliczenia wartości gier i optymalnych strategii wykorzystamy funkcję *game_value_and_strategy()*, która przyjmuje macierz wypłat zgodnie ze wzorem ze wspomnianego artykułu, następnie generuje problem liniowy, by obliczyć wartość gry, z czego jednocześnie otrzymujemy optymalne strategie. 

Skorzystamy jeszcze z funkcji pomocniczej *list_split()*, która przyjmuje listę, a następnie przekształca ją na format, który pozwala łatwo wygenerować obiekt *matrix* pakietu *SageMath*. 

In [1]:
from itertools import combinations

In [2]:
import math

def list_split(lista):
     # sprawdzenie czy pierwiastek kwadratowy z długości listy jest liczbą całkowitą
    if math.sqrt(len(lista)) % 1 == 0:
        n = int(math.sqrt(len(lista)))  # obliczenie pierwiastka kwadratowego z długości listy
        return [lista[i * n:(i + 1) * n] for i in range(n)]  # podzielenie listy na podlisty
    else:
        return "Długość listy nie jest kwadratem liczby naturalnej."

In [3]:
# przykład działania funkcji list_split()
lista = [1, 2, 3, 4, 5, 6, 7, 8, 9]
print(list_split(lista))

# tak ta funkcja jest napisana, żeby "wpasować się" w składnię SageMath
macierz = matrix(list_split(lista)) 
show(macierz)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


[1 2 3]
[4 5 6]
[7 8 9]

In [4]:
from sage.numerical.interactive_simplex_method import *

def game_value_and_strategy(payoff_matrix):
    m, n = payoff_matrix.dimensions()
    A = matrix(QQ, m + 1, n + 1)
    A[0:m, 0:n] = payoff_matrix.transpose() # transponujemy macierz, żeby zachować perspektywę gracza wierszowego
    A[0:m, n] = -1
    A[m, 0:n] = 1
    A[m, n] = 0
    
    b = vector(QQ, m + 1)
    b[m] = 1
    c = vector(QQ, n + 1)
    c[n] = 1
    
    x_names = ["x" + str(i+1) for i in range(n)] + ["x" + str(n+1)]
    constraint_type = [">=" for _ in range(m)] + ["=="]
    variable_type = [">=" for _ in range(n)] + [""] # zmiana względem wersji pierwotnej
    
    P = InteractiveLPProblem(A, b, c, x_names, constraint_type=constraint_type, variable_type=variable_type)
    
    
    optimal_solution = P.optimal_solution()
    objective_value = c * vector(optimal_solution)
    
    return objective_value, optimal_solution

In [5]:
def game_size_k(k: int, f: dict, N: int):
    """
     za k przyjmujemy rozmiar gry -- to znaczy moc zbioru kart w mojej ręcę czy ręcę przeciwnika;
     f będzie słownikiem, do którego będziemy zapisywać wartości gier mniejszych rozmiarów,
     by wykorzystać je do dalszych obliczeń;
     N to liczba kart, którą będziemy grać -- np. gdy N = 3, to oznacza, że gramy Asem, dwójką i trójką.
     """
    
    if k < 0:
        return 0
    
    elif k == 1:
        for A in combinations(range(1, N+1), k):
            for B in combinations(range(1, N+1), k):
                for C in combinations(range(1, N+1), k):
                    a = A[0]
                    b = B[0]
                    c = C[0]
                    if a > b:
                        f[A, B, C] = c
                    elif a < b:
                        f[A, B, C] = -c
                    else:
                        f[A, B, C] = 0
    
    else: 
        for A in combinations(range(1, N+1), k):
            for B in combinations(range(1, N+1), k):
                for C in combinations(range(1, N+1), k):
                    sub_sums = 0
                    for c in C:
                        # gra z wyłożoną kartą c
                        payoff_listed = [] # używamy listy zamiast słownika -- jakie to ma korzyści? 
                        for a in A:
                            for b in B:
                                if a > b:
                                    p = c
                                elif a < b: 
                                    p = -c
                                else:
                                    p = 0
                                    
                                As = tuple(sorted(set(A) - {a}))
                                Bs = tuple(sorted(set(B) - {b}))
                                Cs = tuple(sorted(set(C) - {c}))
                                payoff_listed.append(p + f[As, Bs, Cs])
                                
                        # tworzymy macierz sage wykorzystując naszą funkcję list_split
                        payoff_matrix = matrix(list_split(payoff_listed))
                        print(f'Przy A: {A}, B: {B}, C: {C} i wyłożonej karcie: {c} wartość gry to {round(float(game_value_and_strategy(payoff_matrix)[0]), 4)}.')
                        
                        # zamieniamy obiekt vector klasy sage na listę, by móc wypisać strategię
                        strategies = list(game_value_and_strategy(payoff_matrix)[1][:-1]) 
                        print(f'Optymalna strategia gracza pierwszego: {list(map(lambda x: round(float(x), 4), strategies))}')
                        
                        # sumujemy wartości gier po c z C
                        sub_sums += round(float(game_value_and_strategy(payoff_matrix)[0]), 4) 
                        print("\n")
                    f[A, B, C] = round((1 / len(C))*sub_sums, 4)
                    print(f'Wartość oczekiwana dla gry  {f[A, B, C]}.')
                    print("\n")

In [6]:
f = dict()

N = 6

In [8]:
game_size_k(1, f, N)
game_size_k(2, f, N)
game_size_k(3, f, N)
game_size_k(4, f, N)
game_size_k(5, f, N)

In [13]:
game_size_k(6, f, N)

Przy A: (1, 2, 3, 4, 5, 6), B: (1, 2, 3, 4, 5, 6), C: (1, 2, 3, 4, 5, 6) i wyłożonej karcie: 1 wartość gry to 0.0.
Optymalna strategia gracza pierwszego: [0.165, 0.5774, 0.2576, 0.0, 0.0, 0.0]


Przy A: (1, 2, 3, 4, 5, 6), B: (1, 2, 3, 4, 5, 6), C: (1, 2, 3, 4, 5, 6) i wyłożonej karcie: 2 wartość gry to 0.0.
Optymalna strategia gracza pierwszego: [0.0, 0.3253, 0.1814, 0.4933, 0.0, 0.0]


Przy A: (1, 2, 3, 4, 5, 6), B: (1, 2, 3, 4, 5, 6), C: (1, 2, 3, 4, 5, 6) i wyłożonej karcie: 3 wartość gry to 0.0.
Optymalna strategia gracza pierwszego: [0.0655, 0.1314, 0.17, 0.2907, 0.3424, 0.0]


Przy A: (1, 2, 3, 4, 5, 6), B: (1, 2, 3, 4, 5, 6), C: (1, 2, 3, 4, 5, 6) i wyłożonej karcie: 4 wartość gry to 0.0.
Optymalna strategia gracza pierwszego: [0.098, 0.0458, 0.1734, 0.0, 0.6081, 0.0746]


Przy A: (1, 2, 3, 4, 5, 6), B: (1, 2, 3, 4, 5, 6), C: (1, 2, 3, 4, 5, 6) i wyłożonej karcie: 5 wartość gry to 0.0.
Optymalna strategia gracza pierwszego: [0.0273, 0.0865, 0.0, 0.3461, 0.0202, 0.52]


Przy A: 

Otrzymane strategie doskonale odpowiadają temu, co przedstawione jest na stronie http://gcrhoads.byethost4.com/gops.html?i=2

## Napotkane problemy

1. Trudność z implementacją funkcji game_size_k(). Były to problemy natury programistycznej; na samym początku trudno było wybrać odpowiednie podejście do tego, jak zbierać odpowiednie dane i przekazywać je do problemu liniowego. Zdecydowaliśmy korzystać z modułu *InteractiveLPProblem*, co wymagało podania macierzy problemu z góry. Jednak ten problem został rozwiązany poprzez napisanie odpowiedniej funkcji pomocniczej. 

2. Nieprawidłowe sformułowanie problemu liniowego. Podczas pierwszych prób wyświetlał się błąd, który sugerował, że funkcja licząca problem liniowy nie znajduje żadnego rozwiązania optymalnego. Okazało się, że wynikało to z faktu, że niewłaściwie ograniczyliśmy zmienne problemu. Dodawana zmienna x_n+1, która miała być tą maksymalizowaną w problemie, była ustalana na większą lub równą zero, co w sposób oczywisty w wielu przypadkach dawało sprzeczność całego układu. Usunięcie tego ograniczenia rozwiązało problem. 

3. Niewłaściwie rzutowanie obiektu klasy Sage. Przy pierwszych obliczeniach łatwo było zauważyć, że obliczenia optymalnej strategii bardzo znacząco odbiegały od tego, co można było zobaczyć w tabeli na stronie. Okazało się, że nieumyślnie rzutowałem obiekt klasy *Sage* (w szczególności: wektor optymalnych strategii i wartość gry) na *Integer*. Wymagana była drobna poprawka -- wystarczyło rzutować na float z dokładnością do 4 miejsc po przecinku, by uzyskać niezbędne rezultaty. 